In [1]:
import pandas as pd
import numpy as np
import random

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner

import rdkit
from rdkit.Chem import AllChem, rdFingerprintGenerator
from rdkit import Chem, DataStructs

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

import matplotlib.pyplot as plt

import catboost as cat

c:\Users\BSH\.virtualenvs\신약개발-mImSxWNo\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
RANDOM_SEED = 777

In [5]:
def feature_engineer():
  train_df = pd.read_csv("../data/train.csv")
  test_df = pd.read_csv("../data/test.csv")
  train_df = train_df.dropna()
  test_df["AlogP"] = np.where(pd.isna(test_df["AlogP"]), test_df["LogD"], test_df["AlogP"])
  
  train_df["mol"] = train_df["SMILES"].apply(lambda x: Chem.MolFromSmiles(x))
  test_df["mol"] = test_df["SMILES"].apply(lambda x: Chem.MolFromSmiles(x))
  
  train_df['mol'] = train_df['mol'].apply(lambda x: Chem.AddHs(x))
  train_df['num_of_atoms'] = train_df['mol'].apply(lambda x: x.GetNumAtoms())
  train_df['num_of_heavy_atoms'] = train_df['mol'].apply(lambda x: x.GetNumHeavyAtoms())
  
  test_df['mol'] = test_df['mol'].apply(lambda x: Chem.AddHs(x))
  test_df['num_of_atoms'] = test_df['mol'].apply(lambda x: x.GetNumAtoms())
  test_df['num_of_heavy_atoms'] = test_df['mol'].apply(lambda x: x.GetNumHeavyAtoms())
  
  # 중복제거
  def canonize(mol):
    return Chem.MolToSmiles(Chem.MolFromSmiles(mol), isomericSmiles=True, canonical=True)

  canon_smile = []
  for molecule in train_df['SMILES']:
    canon_smile.append(canonize(molecule))
  
  train_df['canon_smiles'] = canon_smile
  
  ind = train_df.index[train_df['canon_smiles'].duplicated()]
  train_df = train_df.drop(ind)
  
  train_df.drop(columns=["id", "SMILES"], inplace=True)
  test_df.drop(columns=["id", "SMILES"], inplace=True)
  
  fmgen = rdFingerprintGenerator.GetMorganGenerator()
  train_fps = train_df["mol"].apply(lambda x: fmgen.GetFingerprintAsNumPy(x))
  train_fps = np.stack(train_fps)
  test_fps = test_df["mol"].apply(lambda x: fmgen.GetFingerprintAsNumPy(x))
  test_fps = np.stack(test_fps)
  
  origin_train_features = train_df[["AlogP", "Molecular_Weight", "Num_H_Acceptors", "Num_H_Donors", "Num_RotatableBonds", "LogD", "Molecular_PolarSurfaceArea", "num_of_atoms", "num_of_heavy_atoms"]].values
  origin_test_features = test_df[["AlogP", "Molecular_Weight", "Num_H_Acceptors", "Num_H_Donors", "Num_RotatableBonds", "LogD", "Molecular_PolarSurfaceArea", "num_of_atoms", "num_of_heavy_atoms"]].values
  
  train_features = np.append(origin_train_features, train_fps, axis=1)
  test_features = np.append(origin_test_features, test_fps, axis=1)
  target = train_df["MLM"].values
  
  return train_features, target, test_features
  

In [6]:
train_x, train_y, test_x = feature_engineer()

In [11]:
def create_model(param):
  return cat.CatBoostRegressor(
    random_state=RANDOM_SEED,
    verbose=False,
    **param
  )

In [12]:
def train_model(param, X, y, X_test, trial=None):
    skf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

    val_scores = []
    y_tests = []
    models = []

    for idx, (train_idx, val_idx) in enumerate(skf.split(X, y)):

        X_train, y_train = X[train_idx], y[train_idx]
        X_val, y_val = X[val_idx], y[val_idx]

        model = create_model(param)
        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], early_stopping_rounds=50, verbose=500)

        y_hat_val = model.predict(X_val)
        score = mean_squared_error(y_val, y_hat_val, squared=True)
        val_scores.append(score)
        print(f'Fold: {idx+1}/5 score = {score:.5f}')

        y_tests.append(model.predict(X_test))
        models.append(model)

        if trial:
            trial.report(score, idx)

            if trial.should_prune():
                raise optuna.TrialPruned()

    return val_scores, y_tests, models

In [13]:
def objective_cat(trial):
    param = {
        "iterations": trial.suggest_int("iterations", 2000, 20000),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3),
        "depth": trial.suggest_int("depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.1, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
    }

    val_scores, y_tests, models = train_model(param, train_x, train_y, test_x, trial)

    return sum(val_scores) / len(val_scores)

In [14]:
study = optuna.create_study(
    sampler=TPESampler(seed=RANDOM_SEED),
    direction='minimize',
    study_name='cat_tuning',
    pruner=HyperbandPruner(
        min_resource=1, max_resource=8, reduction_factor=3
    ),
)

study.optimize(objective_cat, n_trials=70)

[I 2023-09-07 05:50:37,038] A new study created in memory with name: cat_tuning


0:	learn: 35.3117057	test: 35.3117057	test1: 35.1081674	best: 35.1081674 (0)	total: 149ms	remaining: 11m 46s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 30.82722007
bestIteration = 265

Shrink model to first 266 iterations.
Fold: 1/5 score = 950.31751
0:	learn: 35.1748445	test: 35.1748445	test1: 35.7542603	best: 35.7542603 (0)	total: 1.45ms	remaining: 6.89s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 32.28756767
bestIteration = 351

Shrink model to first 352 iterations.
Fold: 2/5 score = 1042.48703
0:	learn: 35.3284797	test: 35.3284797	test1: 35.0380499	best: 35.0380499 (0)	total: 1.51ms	remaining: 7.16s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 31.14080524
bestIteration = 441

Shrink model to first 442 iterations.
Fold: 3/5 score = 969.74976
0:	learn: 35.2970261	test: 35.2970261	test1: 35.2642561	best: 35.2642561 (0)	total: 1.59ms	remaining: 7.56s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 31.08

[I 2023-09-07 05:50:54,805] Trial 0 finished with value: 983.9140101627866 and parameters: {'iterations': 4748, 'learning_rate': 0.09140462630254884, 'depth': 1, 'subsample': 0.5138743074480719, 'min_data_in_leaf': 86, 'colsample_bylevel': 0.9306471953623999}. Best is trial 0 with value: 983.9140101627866.


Fold: 5/5 score = 990.72827
0:	learn: 34.8131470	test: 34.8131470	test1: 34.5832816	best: 34.5832816 (0)	total: 3.18ms	remaining: 48s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 30.94997222
bestIteration = 80

Shrink model to first 81 iterations.
Fold: 1/5 score = 957.90079
0:	learn: 35.1895544	test: 35.1895544	test1: 35.8928458	best: 35.8928458 (0)	total: 3.21ms	remaining: 48.4s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 32.15339829
bestIteration = 53

Shrink model to first 54 iterations.
Fold: 2/5 score = 1033.84102
0:	learn: 35.4897466	test: 35.4897466	test1: 35.3533796	best: 35.3533796 (0)	total: 3.15ms	remaining: 47.4s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 31.32868432
bestIteration = 79

Shrink model to first 80 iterations.
Fold: 3/5 score = 981.48647
0:	learn: 35.4630042	test: 35.4630042	test1: 35.5640722	best: 35.5640722 (0)	total: 3.52ms	remaining: 53.1s
Stopped by overfitting detector  (50 iterations wait

[I 2023-09-07 05:51:08,484] Trial 1 pruned. 


...

0:	learn: 35.6793129	test: 35.6793129	test1: 35.4857628	best: 35.4857628 (0)	total: 88.1ms	remaining: 28m 18s
500:	learn: 29.1734279	test: 29.1734279	test1: 31.0236430	best: 31.0228468 (499)	total: 46.2s	remaining: 28m 50s
1000:	learn: 26.5741504	test: 26.5741504	test1: 30.6005852	best: 30.6005852 (1000)	total: 1m 31s	remaining: 27m 52s
1500:	learn: 22.9928731	test: 22.9928731	test1: 30.3584041	best: 30.3584041 (1500)	total: 2m 16s	remaining: 27m
Stopped by overfitting detector  (50 iterations wait)

bestTest = 30.29534793
bestIteration = 1771

Shrink model to first 1772 iterations.
Fold: 1/5 score = 917.80811
0:	learn: 35.5276251	test: 35.5276251	test1: 36.0851670	best: 36.0851670 (0)	total: 100ms	remaining: 32m 12s
500:	learn: 28.6734949	test: 28.6734949	test1: 32.4744563	best: 32.4744563 (500)	total: 51.4s	remaining: 32m 4s
1000:	learn: 26.1775161	test: 26.1775161	test1: 32.1978002	best: 32.1969681 (996)	total: 1m 43s	remaining: 31m 27s
1500:	learn: 22.8316333	test: 22.8316333	test1

[I 2023-09-07 12:05:57,952] Trial 68 finished with value: 948.2036996843902 and parameters: {'iterations': 19280, 'learning_rate': 0.0074283984601814495, 'depth': 10, 'subsample': 0.6707649025673613, 'min_data_in_leaf': 72, 'colsample_bylevel': 0.4138112207751321}. Best is trial 64 with value: 946.7478909680689.


Fold: 5/5 score = 937.68191
0:	learn: 35.6801833	test: 35.6801833	test1: 35.4835739	best: 35.4835739 (0)	total: 97.6ms	remaining: 30m 59s
500:	learn: 28.8582030	test: 28.8582030	test1: 30.9467507	best: 30.9461645 (499)	total: 46.3s	remaining: 28m 34s
1000:	learn: 26.0631324	test: 26.0631324	test1: 30.5849713	best: 30.5825224 (998)	total: 1m 32s	remaining: 27m 47s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 30.35234526
bestIteration = 1421

Shrink model to first 1422 iterations.
Fold: 1/5 score = 921.26487
0:	learn: 35.5228799	test: 35.5228799	test1: 36.0781459	best: 36.0781459 (0)	total: 97.3ms	remaining: 30m 52s
500:	learn: 28.4158177	test: 28.4158177	test1: 32.3960760	best: 32.3954496 (498)	total: 51.9s	remaining: 32m 1s
1000:	learn: 25.7068175	test: 25.7068175	test1: 32.0959924	best: 32.0954183 (994)	total: 1m 44s	remaining: 31m 20s
1500:	learn: 22.0089658	test: 22.0089658	test1: 31.8125593	best: 31.8117410 (1495)	total: 2m 36s	remaining: 30m 31s
2000:	learn: 1

[I 2023-09-07 12:23:01,358] Trial 69 finished with value: 946.5019472442415 and parameters: {'iterations': 19047, 'learning_rate': 0.00803959943414251, 'depth': 10, 'subsample': 0.6147089339424189, 'min_data_in_leaf': 74, 'colsample_bylevel': 0.41237498315699683}. Best is trial 69 with value: 946.5019472442415.


Fold: 5/5 score = 927.55574


In [15]:
trial = study.best_trial

In [16]:
print(trial.params)

{'iterations': 19047, 'learning_rate': 0.00803959943414251, 'depth': 10, 'subsample': 0.6147089339424189, 'min_data_in_leaf': 74, 'colsample_bylevel': 0.41237498315699683}


In [17]:
best_param = {
  'iterations': 19047,
  'learning_rate': 0.00803959943414251,
  'depth': 10,
  'subsample': 0.6147089339424189,
  'min_data_in_leaf': 74,
  'colsample_bylevel': 0.41237498315699683
}
val_scores, y_tests, models = train_model(best_param, train_x, train_y, test_x)

0:	learn: 35.6801833	test: 35.6801833	test1: 35.4835739	best: 35.4835739 (0)	total: 110ms	remaining: 34m 50s
500:	learn: 28.8582030	test: 28.8582030	test1: 30.9467507	best: 30.9461645 (499)	total: 50.9s	remaining: 31m 25s
1000:	learn: 26.0631324	test: 26.0631324	test1: 30.5849713	best: 30.5825224 (998)	total: 1m 38s	remaining: 29m 37s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 30.35234526
bestIteration = 1421

Shrink model to first 1422 iterations.
Fold: 1/5 score = 921.26487
0:	learn: 35.5228799	test: 35.5228799	test1: 36.0781459	best: 36.0781459 (0)	total: 132ms	remaining: 41m 49s
500:	learn: 28.4158177	test: 28.4158177	test1: 32.3960760	best: 32.3954496 (498)	total: 57.5s	remaining: 35m 28s
1000:	learn: 25.7068175	test: 25.7068175	test1: 32.0959924	best: 32.0954183 (994)	total: 1m 53s	remaining: 33m 58s
1500:	learn: 22.0089658	test: 22.0089658	test1: 31.8125593	best: 31.8117410 (1495)	total: 2m 49s	remaining: 32m 59s
2000:	learn: 19.0678742	test: 19.0678742	te

In [12]:
for idx, model in enumerate(models):
  model.save_model(f"model/catboost_MLM_fold_v1_{idx}.model")

In [13]:
submit = pd.read_csv("./catboost_optuna.csv")
submit["MLM"] = np.mean(y_tests, axis=0)
submit.to_csv("catboost_optuna.csv", index=False)